In [1]:
import gdal
import os
import pandas as pd

# Declare user inputs

## Declare the ECOSG original path

In [2]:
input_ecosgcoverpath='/data/gbessardon/ECOCLIMAP-SG/COVER/ecosg_final_map.dir'
outputfn='ecosg_chunk.dir'

## Declare new bounds

In [3]:
xmin=-180
xmax=180
ymin=0
ymax=90

# Declare functions

## Prepare the cover file

### Create python readable .hdr 

In [4]:
def create_readablehdr(fnhdr,outfn):

    dictecosg=pd.read_csv(fnhdr,delimiter=":",
                      names=['characteristics','value'],
                      index_col='characteristics',
                     skiprows=1).to_dict()['value']

    dictp={'nrows' : '16800',
           'ncols' : '43200',
           'nodata' : '0',
           'nbands' : '1',
           'nbits'  : '8',
           'ULXMAP' :    '-180.00000000000000',
           'ULYMAP' :      '79.9999999999986',
           'XDIM'  :        '0.00277777777777',
           'YDIM'  :        '0.00277777777777'}
    
    dictp['nodata']=dictecosg['nodata']
    dictp['nrows']=dictecosg['rows']
    dictp['ncols']=dictecosg['cols']
    dictp['ULXMAP']=dictecosg['west']
    dictp['ULYMAP']=dictecosg['north']
    dictp['XDIM']=str(abs((float(dictecosg['west'])-float(dictecosg['east']))/float(int(dictecosg['cols']))))
    dictp['YDIM']=str(abs((float(dictecosg['north'])-float(dictecosg['south']))/float(int(dictecosg['rows']))))
    with open(outfn,'w+') as f:
        for i,v in dictp.items():
            f.write(i+' '+v+'\n')
        f.close()
    return(dictp,dictecosg)

### Create a soft link to ECOSG .dir file and create a python readable .hdr file

In [5]:
def Create_readable_ecosg_cover(input_ecosgcoverpath):
    fnlink=os.path.join(os.getcwd(),input_ecosgcoverpath.split('/')[-1])
    if not os.path.exists(fnlink):
        os.symlink(input_ecosgcoverpath,os.path.join(os.getcwd(),input_ecosgcoverpath.split('/')[-1]))
    fnhdr=input_ecosgcoverpath.replace('.dir','.hdr')    
    outfn=fnlink.replace('.dir','.hdr') 
    (dictp,dictecosg)=create_readablehdr(fnhdr,outfn)
    return(dictp,dictecosg,outfn,fnlink)
    

## All together

In [6]:
def Crop_cover(input_ecosgcoverpath,xmin, ymin, xmax, ymax):
    _,_,_,fnlink=Create_readable_ecosg_cover(input_ecosgcoverpath)
    opt=gdal.WarpOptions(format='EHdr',outputBounds=(xmin, ymin, xmax, ymax))
    gdal.Warp(outputfn,fnlink,options=opt)
    return

# MAIN

In [7]:
Crop_cover(input_ecosgcoverpath,xmin, ymin, xmax, ymax)

OSError: [Errno 122] Disk quota exceeded

In [ ]:
fnhdr=outputfn.replace('.dir','.hdr')

In [ ]:
    f=open(fnhdr,'r')
    lines=f.readlines()
    characteristics=[l.split(' ')[0] for l in lines]
    value=[l.strip().replace('  ',' ').split(' ')[1] for l in lines]
    dicti ={'characteristics':characteristics, 'value':value}
    dictuncompress=pd.DataFrame(dicti,index=characteristics).to_dict()['value']
    dictp={'ECOCLIMAP' : '',
            'nodata' : '0',
            'north' : '80.',
            'south' : '-60.',
            'west' : '-180.',
            'east' : '180.',
            'rows' : '50400',
            'cols' : '129600',
            'fact' : '10',
            'compress' : '1',
            'recordtype' : 'integer 16 bytes' }
    dictp['nodata']=dictuncompress['nodata']
    dictp['rows']=dictuncompress['nrows']
    dictp['cols']=dictuncompress['ncols']
    dictp['north']=dictuncompress['ULYMAP']
    dictp['south']=str(float(dictuncompress['ULYMAP'])-(float(dictuncompress['nrows'])*float(dictuncompress['YDIM'])))
    dictp['west']=dictuncompress['ULXMAP']
    dictp['east']=str(float(dictuncompress['ULXMAP'])+(float(dictuncompress['ncols'])*float(dictuncompress['XDIM'])))
    with open(outfn,'w+') as f:
        for i,v in dictp.items():
            f.write(i+': '+v+'\n')
        f.close()